In [1]:
import gurobipy as gp
from gurobipy import GRB
import json


def solve_dsn_scheduling(graph, I, A, T, S, e_iw, l_iw, W, M, start_job,end_job ):
    """
    Solve the DSN scheduling problem using Gurobi.

    Parameters:
    graph (SmartGraph): Graph representing the scheduling problem
    I (list): Set of jobs
    A (list): Set of antennas
    T (int): Total time period
    S (list of tuples): Set of synchronous job pairs
    e_iw (dict): Earliest start time for job i in time window w
    l_iw (dict): Latest end time for job i in time window w
    M (int): A sufficiently large constant

    Returns:
    dict: Solution containing variable values if optimal solution is found, else None
    """

    # Print each parameter received by the function
    print("Graph Nodes:", graph.nodes)
    print("Set of jobs (I):", I)
    print("Set of antennas (A):", A)
    print("Total time period (T):", T)
    print("Set of synchronous job pairs (S):", S)
    print("Earliest start time for job i in time window w (e_iw):", e_iw)
    print("Latest end time for job i in time window w (l_iw):", l_iw)
    print("Dictionary of time windows for each job i (W):", W)
    print("A sufficiently large constant (M):", M)
    print("Start job node:", start_job)
    print("End job node:", end_job)

    print(W)
    # Extract nodes and edges from the graph
    V = graph.nodes
    E = [(u.track_id, v.track_id) for u in V for v, _ in graph.get_edges(u)]
    print(E)
    s = {node: node.duration for node in V}
    s_uv = {(u.track_id, v.track_id): weight for u in V for v, weight in graph.get_edges(u)}

    # Create a new model
    model = gp.Model("DSN_Scheduling")

    # Decision variables
    x = model.addVars(set(E), A, vtype=GRB.BINARY, name="x")
    z = model.addVars(V, W, A, vtype=GRB.BINARY, name="z")
    y = model.addVars(I, W, vtype=GRB.BINARY, name="y")
    w = model.addVars(V, A, vtype=GRB.CONTINUOUS, name="w")

    # Objective: Maximize the number of successfully scheduled jobs
    model.setObjective(gp.quicksum(x[u, v, a] for (u, v) in E for a in A), GRB.MAXIMIZE)

    # Constraints
    # Start Constraint
    model.addConstrs((gp.quicksum(x[start_job, v, a] for v in V if (start_job, v) in E) == 1 for a in A), name="start")

    # Flow Conservation Constraint
    model.addConstrs((gp.quicksum(x[u, v, a] for v in V if (u, v) in E) ==
                     gp.quicksum(x[v, u, a] for v in V if (v, u) in E)
                     for u in V for a in A), name="flow_conservation")

    # End Constraint
    model.addConstrs((gp.quicksum(x[u, end_job, a] for u in V if (u, end_job) in E) <= 1 for a in A), name="end")

    # Viewpoint Selection Constraint
    model.addConstrs((gp.quicksum(x[u, v, a] for u in V if (u, v) in E) ==
                     gp.quicksum(z[v, w, a] for w in W)
                     for v in V for a in A), name="viewpoint_selection")
    
    # Viewpoint Selection Constraint
    model.addConstrs((gp.quicksum(z[v, w, a] for w in W) <= 1
                     for v in V for a in A), name="viewpoint_selection")

    # Time Window Selection Constraint
    model.addConstrs((gp.quicksum(y[i, w] for w in W[i]) <= 1 for i in I), name="sync_1")
 
    # Time Window Availability Constraint
    model.addConstrs((z[u, w, a] <= y[i, w]
                    for i in I
                    for u in V if u == i  # Ensure u is a node associated with job i
                    for w in W[i]
                    for a in A), name="sync_2")

    # Time Window Synchronization Constraint
    model.addConstrs((z[u, w, a] >= y[i, w] + gp.quicksum(x[u, v, a] for v in V if (u, v) in E) - 1
                    for i in I
                    for u in V if u == i  # Ensure u is a node associated with job i
                    for w in W[i]
                    for a in A), name="sync_3")
    
    # Time Continuity Constraint
    model.addConstrs((w[v, a] >= w[u, a] + s[u] + s_uv[u, v] + M * (x[u, v, a] - 1)
                      for u in V for v in V if (u, v) in E for a in A), name="time_window_1")

    # Earliest Start Time Constraint
    model.addConstrs((w[u, a] >= e_iw[i, w] + M * (z[u, w, a] - 1)
                    for i in I
                    for u in V if u == i  # Ensure u is a node associated with job i
                    for w in W[i]
                    for a in A), name="time_window_2")

    # Latest End Time Constraint
    model.addConstrs((w[u, a] + s[u] <= l_iw[i, w] + M * (1 - z[u, w, a])
                    for i in I
                    for u in V if u == i  # Ensure u is a node associated with job i
                    for w in W[i]
                    for a in A), name="time_window_3")

    # Synchronous Jobs Constraint
    #model.addConstrs((w[u, a] == w[u, a_]
    #              for u in V if u in I  # Ensure u is a node associated with job i
    #              for a in A
    #              for a_ in A if a != a_), name="synchronous_jobs")

    # Synchronize Time Windows Across Antennas Constraint
    #model.addConstrs((y[i, w] == y[i_, w] for (i, i_) in S for w in W[i]), name="synchronize_antennas")

    # Time Limit Constraint
    model.addConstrs((w[u, a] <= T * gp.quicksum(x[u, v, a] for v in V if (u, v) in E)
                      for u in V for a in A), name="time_limit")

    # Optimize model
    model.optimize()

    # Retrieve results
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found")
        solution = {}
        for v in model.getVars():
            if v.x > 0:
                solution[v.varName] = v.x
                print(f'{v.varName}: {v.x}')
        return solution
    else:
        print("No optimal solution found")
        return None



class ViewPeriod:
    def __init__(self):
        self.rise = None
        self.set = None
        self.trx_on = None
        self.trx_off = None

class Job:
    def __init__(self):
        self.track_id = ""
        self.subject = 0
        self.week = 0
        self.year = 0
        self.duration = 0.0
        self.duration_min = 0.0
        self.setup_time = 0
        self.teardown_time = 0
        self.time_window_start = 0
        self.time_window_end = 0
        self.antenna_view_periods = {}

def add_view_periods_to_job(job, antenna_name, vp_list):
    for vp in vp_list:
        view_period = ViewPeriod()
        if "RISE" in vp: view_period.rise = vp["RISE"]
        if "SET" in vp: view_period.set = vp["SET"]
        if "TRX ON" in vp: view_period.trx_on = vp["TRX ON"]
        if "TRX OFF" in vp: view_period.trx_off = vp["TRX OFF"]
        if antenna_name not in job.antenna_view_periods:
            job.antenna_view_periods[antenna_name] = []
        job.antenna_view_periods[antenna_name].append(view_period)

def read_jobs_from_file(filename, jobs):
    with open(filename, 'r') as file:
        j = json.load(file)

    for item_key, item_value in j.items():
        if item_key != "W10_2018":
            continue
        for req in item_value:
            job = Job()
            job.track_id = req["track_id"]
            job.subject = req["subject"]
            job.week = req["week"]
            job.year = req["year"]
            job.duration = req["duration"]
            job.duration_min = req["duration_min"]
            job.setup_time = req["setup_time"]
            job.teardown_time = req["teardown_time"]
            job.time_window_start = req["time_window_start"]
            job.time_window_end = req["time_window_end"]

            for antenna_name, vp_list in req["resource_vp_dict"].items():
                if '_' in antenna_name:
                    single_antennas = antenna_name.split('_')
                    for single_antenna in single_antennas:
                        add_view_periods_to_job(job, single_antenna, vp_list)
                else:
                    add_view_periods_to_job(job, antenna_name, vp_list)

            jobs.append(job)

def distribute_jobs_to_antennas(jobs):
    antenna_jobs = {}

    for job in jobs:
        # print(f"Distributing job {job.track_id} to antennas")
        for antenna, view_periods in job.antenna_view_periods.items():
            # print(f"Antenna {antenna}")
            if antenna not in antenna_jobs:
                antenna_jobs[antenna] = []
            antenna_jobs[antenna].append(job)

    return antenna_jobs
class SmartGraph:
    def __init__(self):
        self.nodes = []
        self.edges = {}

    def add_node(self, node):
        self.nodes.append(node)
        self.edges[node] = []

    def add_edge(self, from_node, to_node, weight):
        self.edges[from_node].append((to_node, weight))

    def get_edges(self, node):
        return self.edges[node]

def create_graph(antenna, antenna_jobs):
    start_job = Job()
    start_job.track_id = "start"
    start_job.setup_time = 0
    start_job.duration = 0
    start_job.time_window_start = 0
    start_job.time_window_end = 0
    start_job.antenna_view_periods[antenna] = [ViewPeriod()]
    start_job.antenna_view_periods[antenna][0].rise = 0
    start_job.antenna_view_periods[antenna][0].set = 0
    start_job.antenna_view_periods[antenna][0].trx_on = 0
    start_job.antenna_view_periods[antenna][0].trx_off = 0
    if antenna not in antenna_jobs:
        antenna_jobs[antenna] = []
    antenna_jobs[antenna].append(start_job)

    end_job = Job()
    end_job.track_id = "end"
    end_job.setup_time = 0
    end_job.duration = 0
    end_job.time_window_start = 10900
    end_job.time_window_end = 10900
    end_job.antenna_view_periods[antenna] = [ViewPeriod()]
    end_job.antenna_view_periods[antenna][0].rise = 10900
    end_job.antenna_view_periods[antenna][0].set = 10900
    end_job.antenna_view_periods[antenna][0].trx_on = 10900
    end_job.antenna_view_periods[antenna][0].trx_off = 10900
    antenna_jobs[antenna].append(end_job)

    graph = SmartGraph()
    # Populate the graph with nodes
    for job in antenna_jobs[antenna]:
        graph.add_node(job)

    # Add edges based on view periods and job constraints
    for job1 in antenna_jobs[antenna]:
        for job2 in antenna_jobs[antenna]:
            if job1.track_id == job2.track_id:
                continue  # Skip the same job

            for vp1 in job1.antenna_view_periods[antenna]:
                for vp2 in job2.antenna_view_periods[antenna]:
                    # Logic to determine if a transition from job1 to job2 is feasible
                    if vp2.rise >= vp1.set:  # Example condition: job2 starts after job1 ends
                        transition_time = vp2.rise - vp1.set
                        graph.add_edge(job1, job2, transition_time)

    return graph


jobs = []
read_jobs_from_file("dsn_test_data.json", jobs)
antenna_jobs = distribute_jobs_to_antennas(jobs)
antennas = list(antenna_jobs.keys())

graph = create_graph(antennas[0], antenna_jobs)

I = list(range(len(jobs)))
A = list(range(len(antennas)))

# iterate over jobs and create a dictionary of time windows for each job
W = {}
for i, job in enumerate(jobs):
    # iterate over antenas and view periods
    for antenna, view_periods in job.antenna_view_periods.items():
        for w, view_period in enumerate(view_periods):
            if i not in W:
                W[i] = []
            W[i].append(w)
T = 10  # total time period
S = []  # set of synchronous job pairs
e_iw = {(i, w): jobs[i].time_window_start if i < len(jobs) else 0 for i in I for w in W}  # earliest start times
l_iw = {(i, w): jobs[i].time_window_end if i < len(jobs) else 10 for i in I for w in W}  # latest end times

M = 100  

start_job = 0
end_job = len(graph.nodes) - 1

solution = solve_dsn_scheduling(graph, I, A, T, S, e_iw, l_iw, W, M, start_job,end_job )


Graph Nodes: [<__main__.Job object at 0x7f6f1b1163c0>, <__main__.Job object at 0x7f6f1b116750>, <__main__.Job object at 0x7f6f1b117080>, <__main__.Job object at 0x7f6f1b1171d0>, <__main__.Job object at 0x7f6f1c593950>, <__main__.Job object at 0x7f6f1b1150d0>]
Set of jobs (I): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Set of antennas (A): [0, 1, 2, 3, 4]
Total time period (T): 10
Set of synchronous job pairs (S): []
Earliest start time for job i in time window w (e_iw): {(0, 0): 803, (0, 1): 803, (0, 2): 803, (0, 3): 803, (0, 4): 803, (0, 5): 803, (0, 6): 803, (0, 7): 803, (0, 8): 803, (0, 9): 803, (1, 0): 904, (1, 1): 904, (1, 2): 904, (1, 3): 904, (1, 4): 904, (1, 5): 904, (1, 6): 904, (1, 7): 904, (1, 8): 904, (1, 9): 904, (2, 0): 1705, (2, 1): 1705, (2, 2): 1705, (2, 3): 1705, (2, 4): 1705, (2, 5): 1705, (2, 6): 1705, (2, 7): 1705, (2, 8): 1705, (2, 9): 1705, (3, 0): 4912, (3, 1): 4912, (3, 2): 4912, (3, 3): 4912, (3, 4): 4912, (3, 5): 4912, (3, 6): 4912, (3, 7): 4912, (3, 8): 4912, (3, 9): 49

NameError: name 'pato' is not defined